The following main packages/modules are used:
- pandas
- numpy
- ScikitLearn
- scikit-optimize
- XGBoost
- joblib

In [33]:
import pandas as pd
import numpy as np
from numpy import argmax
from copy import deepcopy
import joblib

from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import average_precision_score
from skopt import BayesSearchCV

import xgboost as xgb
from xgboost import XGBClassifier

## 1. Loading Data

Load the training and testing dataset we created in the feature engineering for training notebook.

In [34]:
# run this cell to load train and test/validation datasets
train = pd.read_csv("train_final.csv", header=None)
test = pd.read_csv("test_final.csv", header=None)

In [35]:
#read words from trained count vectorizer (in feature engineering) to assign to column headers
cv = joblib.load('countvectorizer.pkl')
words = cv.get_feature_names_out()

In [36]:
names = ['victim_age_1','subject_age_1','East', 'North', 'precinct_OOJ', 'South', 'Southwest', 'West', 'precinct_Unknown',
 'Female','Gender Diverse (gender non-conforming and/or transgender)', 'Male', 'Vic_Gender_Unknown',
 'American Indian or Alaska Native', 'Asian', 'Black or African American', 'Native Hawaiian or Other Pacific Islander',
 'Vic_Race_Unknown', 'White', 'Hispanic Or Latino', 'Not Hispanic Or Latino', 'Vic_Ethni_Unknown',
 'subject_American Indian or Alaska Native', 'subject_Asian', 'subject_Black or African American',
 'subject_Native Hawaiian or Other Pacific Islander', 'subject_Sub_Race_Unknown', 'subject_White',
 'subject_Female', 'subject_Gender Diverse (gender non-conforming and/or transgender)',
 'subject_Male', 'subject_Sub_Gender_Unknown', 'subject_Hispanic Or Latino', 'subject_Not Hispanic Or Latino',
 'subject_Sub_Ethni_Unknown', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3', 'D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2',
 'F3', 'G1', 'G2', 'G3', 'H1', 'H2', 'H3', 'J1', 'J2', 'J3', 'K1', 'K2', 'K3', 'L1', 'L2', 'L3', 'M1', 'M2', 'M3',
 'N1', 'N2', 'N3', 'O1', 'O2', 'O3', 'Q1', 'Q2', 'Q3', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3', 'U1', 'U2', 'U3',
 'beat_Unknown', 'W1', 'W2', 'W3', 'beat_OOJ']

col_names = np.concatenate((['label'], words, names))

train = train.rename(columns=dict(zip(train.columns, col_names), inplace=True))
test = test.rename(columns=dict(zip(test.columns, col_names), inplace=True))

We use __[AWS' XGBoost built-in algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html)__, running asynchronous __[hyperparameter tuning jobs](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-ex-tuning-job.html)__ and training jobs using AWS Sagemaker. 

However, for the sake of replication we show how to train your model locally using __[DMLC XGBoost](https://xgboost.readthedocs.io/en/stable/index.html)__. 

In [37]:
#separate label from features

y_train = train.iloc[:,0]
X_train = train.iloc[:,1:]

y_test = test.iloc[:,0]
X_test = test.iloc[:,1:]

Fit the model to obtain the default hyperparameter values before tuning.

In [38]:
xgbd = XGBClassifier()

#fit model
xgbd.fit(X_train, y_train)

#save default parameters
default_params = {}
gparams = xgbd.get_params()

#create lists of param values for tuning
for key in gparams.keys():
    gp = gparams[key]
    default_params[key] = [gp]

In our application, we use Bayesian optimization in a Sagemaker hyperparameter tuning job. For the sake of replication, we show how to tune your model's hyperparameters with scikit-optimize BayesSearchCV.

We heavily borrow the tuning implementation code from https://towardsdatascience.com/binary-classification-xgboost-hyperparameter-tuning-scenarios-by-non-exhaustive-grid-search-and-c261f4ce098d

It is important to reiterate that our application's tuning and training strategies were selected based on the amount of data we work with and other data and computational cost considerations. The datasize in this example is very small, creating issues with overfitting, hyperparameter sensitivity to subsamples and high variance, unreliable performance metrics, as well as poor generalization, which are not usually a concern with larger datasets. 

Again, we provide this code for demonstration purposes. Careful consideration should be given to your application's specific conditions and applicable best practices.

In [39]:
#set ranges for each hyperparameter to be tuned (can be modified as appropriate)

param_grid = {'gamma': [0.1,0.2,0.4,0.8,1.5,3.5,6],
              'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.2, 0.3],
              'max_depth': [5,6,7,8,9,10],
              'n_estimators': [10,20,30,50,80,100],
              'reg_alpha': [0,0.1,0.2,0.4,0.8,1],
              'reg_lambda': [0,0.1,0.2,0.4,0.8,1]}

In [40]:
results_dict = {}

#list values of default parameters
default_params_xgb = {}

for key in default_params.keys():
    default_params_xgb[key] = default_params[key][0]

#providing default parameters to xgbc model
xgbc = xgb.XGBClassifier(**default_params_xgb)

#create estimator (with a larger dataset, consider more iterations; given class imbalance, we optimize the F-1 score)
clf = BayesSearchCV(estimator=xgbc, search_spaces=param_grid, n_iter=5, scoring='f1', cv=3, 
                    return_train_score=True, verbose=0)

clf.fit(X_train, y_train)

#save results to dataframe
df = pd.DataFrame(clf.cv_results_)
    
#predictions
train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)
    
#F1 scores for each train/test label
f1s_train_p1 = f1_score(y_train, train_predictions, pos_label=1)
f1s_train_p0 = f1_score(y_train, train_predictions, pos_label=0)
f1s_test_p1 = f1_score(y_test, test_predictions, pos_label=1)
f1s_test_p0 = f1_score(y_test, test_predictions, pos_label=0)
    
#best parameters
bp = clf.best_params_
    
#storing computed values in results dictionary
results_dict['xgbc_bcv'] = {'classifier': deepcopy(clf),
                            'cv_results': df.copy(),
                            'train F1-score label 1': f1s_train_p1,
                            'train F1-score label 0': f1s_train_p0,
                            'test F1-score label 1': f1s_test_p1,
                            'test F1-score label 0': f1s_test_p0,
                            'best_params': bp}


Refit classifier with best hyperparameters from tuning.

In [41]:
xgbf = xgb.XGBClassifier(**bp)
xgbf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=10, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [42]:
# save trained model
xgbf.save_model('xgboost_model')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [19:25:16] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


## 2. Prediction on test dataset to get optimal classification threshold

We encounter severe class imbalance in our application. We test different strategies to improve the model's performance (not discussed in this example), including selecting a classification threshold that optimizes the performance metric of interest (the F-1 score in our case).

In [43]:
# make predictions
preds_prob = xgbf.predict_proba(X_test)
prediction = xgbf.predict(X_test)

test["prediction"] = prediction
test["pred_prob"] = preds_prob[:,1]

In [45]:
print(confusion_matrix(test.label, prediction))
print(classification_report(test.label, prediction))
print(roc_auc_score(test.label, prediction))

[[56  1]
 [ 5 17]]
              precision    recall  f1-score   support

           0       0.92      0.98      0.95        57
           1       0.94      0.77      0.85        22

    accuracy                           0.92        79
   macro avg       0.93      0.88      0.90        79
weighted avg       0.93      0.92      0.92        79

0.8775917065390749


Optimization code based on: https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/

In [46]:
precision, recall, thresholds = precision_recall_curve(test.label, preds_prob[:,1])

In [47]:
#optimize F-1 score (harmonic mean of precision and recall)

fscore = (2 * precision * recall) / (precision + recall)
ix = argmax(fscore)

print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

Best Threshold=0.474340, F-Score=0.878


In [48]:
preds_0_1_nt = [1 if i >= thresholds[ix] else 0 for i in preds_prob[:,1]]

print(confusion_matrix(test.label, preds_0_1_nt))
print(classification_report(test.label, preds_0_1_nt))
print(roc_auc_score(test.label, preds_0_1_nt))

[[56  1]
 [ 4 18]]
              precision    recall  f1-score   support

           0       0.93      0.98      0.96        57
           1       0.95      0.82      0.88        22

    accuracy                           0.94        79
   macro avg       0.94      0.90      0.92        79
weighted avg       0.94      0.94      0.94        79

0.9003189792663477


In [49]:
#save optimal classification threshold in the desired format
ot = np.array([thresholds[ix]])
np.savetxt('optimal_threshold.csv', ot)